# Базовый модуль

In [2]:
from sklearn.externals import joblib

from keras.models import *
from keras.layers import *
from keras.callbacks import *

from sklearn.metrics import *
from sklearn.cross_validation import train_test_split

## Загрузка данных

In [4]:
data_dir = "../../data/X_Y_data/"

X = joblib.load(data_dir + "res_X_train.pklz")
Y = joblib.load(data_dir + "Y_train.pklz")

### Разбиваем множество на тренировочное и тестировочное

In [14]:
X_test = {}
X_train = {}

random_state = 234
train_inds, test_inds = train_test_split(range(len(Y)), train_size=0.8, random_state=random_state)
Y_train, Y_test = Y[train_inds], Y[test_inds]

for k, v in X.items():
    train_vals, test_vals = np.array(v)[train_inds], np.array(v)[test_inds]
    X_train[k] = train_vals
    X_test[k] = test_vals

In [15]:
# X_test_bank = joblib.load(data_dir + "res_X_test_bank.pklz")
# Y_test_bank = joblib.load(data_dir + "Y_test_bank.pklz")

# X_test_telekom = joblib.load(data_dir + "res_X_test_telekom.pklz")
# Y_test_telekom = joblib.load(data_dir + "Y_test_telekom.pklz")

## Функция для оценки

In [16]:
def f1_for_model(model, list_inp_params, X_test, Y_test):
    pred = model.predict([X_test[val] for val in list_inp_params])
    
    pred_labels = np.argmax(pred, axis=1)
    
    Y_test_labels = np.argmax(Y_test, axis=1)
    
    f1_micro = f1_score(Y_test_labels, pred_labels, average="micro", labels=[0, 2], pos_label=None)
    f1_macro = f1_score(Y_test_labels, pred_labels, average="macro", labels=[0, 2], pos_label=None)
    
    res = {"f1_micro": f1_micro, "f1_macro": f1_macro}
    
    return res

### Список признаков в X

In [17]:
X_train.keys()

['X_d2v',
 'X_sum_ranks',
 'X_ton_dicts',
 'X_w2v_ruscorpora_parent',
 'X_POS',
 'X_w2v_web_parent',
 'X_w2v_wikiruscorpora',
 'X_w2v_web',
 'X_sum_ranks_parent',
 'X_POS_parent',
 'X_ton_dicts_parent',
 'X_links',
 'X_w2v_twitter',
 'X_inds_parent',
 'X_w2v_wikiruscorpora_parent',
 'X_inds',
 'X_d2v_parent',
 'X_w2v_twitter_parent',
 'X_w2v_ruscorpora']

## Базовая модель
Редактировать только:
```
    ...
    
    ### Основная модель (редактировать только эту часть) ###
    
    l_D = Flatten()(l_M)
    
    ### Основная модель ###
    
    ...
```

In [23]:
def build_and_evaluate_model(X_train, list_inp_params, embedding_output_dim, model_optimizer,
                             model_loss, model_batch_size, model_nb_epoch,
                             early_stoping_patience, model_verbose, embedding_input_dim,
                             model_hidden_l_params):
    # Входы модели
    # Если вход Индекс -- то добавляем Embedding слой одинаковый для слова и родителя
    l_inputs = [] # Входы
    l_inp_layers = [] # Первые слои (елси нет Embedding идентичны входам)
    embedding_all_lemms = Embedding(embedding_input_dim, embedding_output_dim, input_length=X_train[list_inp_params[0]].shape[1])
    for val in list_inp_params:
        if val in ["X_inds", "X_inds_parent"]:
            model_inp = Input(shape=X_train[val].shape[1:], dtype="int32")
            model_inp_embedding = embedding_all_lemms(model_inp)
            l_inp_layers.append(model_inp_embedding)
        else:
            model_inp = Input(shape=X_train[val].shape[1:])
            l_inp_layers.append(model_inp)
        l_inputs.append(model_inp)

    # Объединяем всё
    if len(list_inp_params) > 1:
        l_M = merge(l_inp_layers, mode="concat")
    else:
        # Если только 1 вход, то объединение не надо
        l_M = l_inp_layers[0]

    ### Основная модель (редактировать только эту часть) ###
    
    l_D = Flatten()(l_M)
    
    ### Основная модель ###
    
    # Выход
    l_O = Dense(3, activation="softmax")(l_D)
    
    # Компиляция модели
    model = Model(input=l_inputs, output=[l_O])
    model.compile(model_optimizer, model_loss, metrics=["accuracy"])

    # Обучение модели с ранним остановом
    es = EarlyStopping(patience=early_stoping_patience)
    h = model.fit([X_train[val] for val in list_inp_params], Y_train, batch_size=model_batch_size, nb_epoch=model_nb_epoch, callbacks=[es], verbose=model_verbose, validation_split=0.3)

    # Проверка модели
    res_f1 = f1_for_model(model, list_inp_params, X_test, Y_test)
    return res_f1, h.history

## Модель 1
Входные признаки:
- word2vec по web корпусу самого слова

In [24]:
# Для сохранения результатов
l_scores = []
model_ind = 0

In [25]:
base_model = "model_1"
list_inp_params = ["X_w2v_web"]

### Модель 1 Обучение и тестирование

In [27]:
# Название модели
model_name = base_model + "_" + "Model_{0}".format(model_ind)
model_ind += 1

# Параметры модели
embedding_output_dim = 45
model_optimizer = "adam"
model_loss = "categorical_crossentropy"
model_batch_size = 100
model_nb_epoch = 100
early_stoping_patience = 1
model_verbose = 0
embedding_input_dim = X_train["X_inds"].max() + 1

model_hidden_l_params = {"GRU_0": 100,
                         "GRU_1": 100,
                         "Dropout": 0.2}

model_params = {"embedding_output_dim": embedding_output_dim,
                "model_optimizer": model_optimizer,
                "model_loss": model_loss,
                "model_batch_size": model_batch_size,
                "model_nb_epoch": model_nb_epoch,
                "early_stoping_patience": early_stoping_patience,
                "model_verbose": model_verbose,
                "embedding_input_dim": embedding_input_dim,
                "model_hidden_l_params": model_hidden_l_params}

for i in range(10):
    f1, h = build_and_evaluate_model(X_train, list_inp_params, embedding_output_dim, model_optimizer,
                                     model_loss, model_batch_size, model_nb_epoch,
                                     early_stoping_patience, model_verbose, embedding_input_dim,
                                     model_hidden_l_params)
    print(f1)
    l_scores.append((model_name, i, f1, h, model_params))

# Дампим результат
# joblib.dump(l_scores, "res_{0}.pklz".format(base_model), compress=3)

{'f1_micro': 0.61411549037580204, 'f1_macro': 0.52820232172093617}
{'f1_micro': 0.61269549218031283, 'f1_macro': 0.53093866333260109}
{'f1_micro': 0.61993627674101048, 'f1_macro': 0.53530568307177684}
{'f1_micro': 0.61839708561020035, 'f1_macro': 0.53418278607905134}
{'f1_micro': 0.61580630424851535, 'f1_macro': 0.53251110828712778}
{'f1_micro': 0.61608775137111516, 'f1_macro': 0.5318645149372786}
{'f1_micro': 0.61801385681293297, 'f1_macro': 0.53211292242093278}
{'f1_micro': 0.61784897025171626, 'f1_macro': 0.52813085339470967}
{'f1_micro': 0.61545538178472858, 'f1_macro': 0.53093068588784631}
{'f1_micro': 0.62306993642143504, 'f1_macro': 0.53790398245341986}


In [28]:
print("Выполнено")

Выполнено
